In [1]:
import numpy as np
import os
import argparse
import time
import math
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn import init
import torch.nn.functional as F
import torch.utils.data as data_utils
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.autograd import Variable
import gzip
import pickle
from scipy import sparse
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.cuda.amp import autocast 
torch.set_default_tensor_type(torch.FloatTensor)




In [2]:
# Load train and test data from files
train_waveforms = np.genfromtxt('Data/trainset.csv', delimiter=',')
train_times = np.genfromtxt('Data/traintime.csv', delimiter=',')
test_waveforms = np.genfromtxt('Data/testset.csv', delimiter=',')
test_times = np.genfromtxt('Data/testtime.csv', delimiter=',')

# Concatenate train and test data along the first axis
all_waveforms = np.concatenate((train_waveforms, test_waveforms), axis=0)
all_times = np.concatenate((train_times, test_times), axis=0)

# Print the shapes of the concatenated datasets
#print('All waveforms:', all_waveforms.shape)
#print('All times:', all_times.shape)

# Reshape the time array to be a column vector
all_times = all_times.reshape(-1, 1)

# Define the threshold for signal detection
#threshold = 0.1

# Apply thresholding to the all_times array         # to convert time into 0s and 1s
#all_times = np.where(all_times > threshold, 1, 0)

# Print the resulting array
#print(all_times)

# Concatenate the waveform and time arrays along the second axis
all_data = np.concatenate((all_waveforms, all_times), axis=1)

# Print the shape of the concatenated dataset
print('All data:', all_data.shape)


All data: (753, 251)


In [3]:
from sklearn.model_selection import train_test_split

In [4]:

# Split the dataset into training set and validation set
X_train, X_val, y_train, y_val = train_test_split(all_waveforms, all_times, test_size=0.2, random_state=42)

In [28]:
# Convert NumPy arrays to PyTorch tensors
X_train = X_train.float()
y_train = y_train.float()
X_val = X_val.float()
y_val = y_val.float()




In [38]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

torch.Size([602, 250])
torch.Size([602, 1])
torch.Size([151, 250])
torch.Size([151, 1])


In [35]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv1d(in_channels=602, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x



In [36]:
# Instantiate the model
model = Net()

# Define an optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Define a loss function
criterion = torch.nn.BCELoss()


num_epochs = 20


# Train the model
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Evaluate the model
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val)
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")





RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x29 and 128x128)